In [4]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer

# Load WikiText-103 dataset
# dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")

datasetDic = load_from_disk("../processed_datadir/wikitext-103-story-bert-512")

In [5]:
print(len(datasetDic["test"]))

62


In [ ]:
for i in range(10):
    print(datasetDic["test"][i]["text"])

In [3]:


# 创建数据集
# dataset = Dataset.from_dict(story_data)

# 保存数据集
# dataset.save_to_disk("../processed_datadir/wikitext-103-story-train")

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("../tokenizer_save/tokenizer-bert-base-uncased-512/")

In [8]:
import multiprocessing

num_proc = multiprocessing.cpu_count() - 4
print(num_proc)
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

def group_texts(examples):
    # print(len(examples["text"]))
    tokenized_inputs = tokenizer(examples["text"], truncation=True,   # 在这就不带文本了
        max_length=tokenizer.model_max_length, return_overflowing_tokens=True,return_length=True)
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []


    
    for length, input_ids in zip(tokenized_inputs["length"], tokenized_inputs["input_ids"]):
        if length == tokenizer.model_max_length:
            input_ids_list.append(input_ids)
        else:
            input_ids_list.append(input_ids + [tokenizer.pad_token_id] * (tokenizer.model_max_length - length))
    for length, token_type_ids in zip(tokenized_inputs["length"], tokenized_inputs["token_type_ids"]):
        if length == tokenizer.model_max_length:
            token_type_ids_list.append(token_type_ids)
        else:
            token_type_ids_list.append(token_type_ids + [0] * (tokenizer.model_max_length - length))
    for length, attention_mask in zip(tokenized_inputs["length"], tokenized_inputs["attention_mask"]):
        if length == tokenizer.model_max_length:
            attention_mask_list.append(attention_mask)
        else:
            attention_mask_list.append(attention_mask + [0] * (tokenizer.model_max_length - length))
    print("success")
    return {"input_ids": input_ids_list, "token_type_ids": token_type_ids_list, "attention_mask": attention_mask_list}

# preprocess dataset
datasetDic["test"] = datasetDic["test"].map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)


12
The max length for the tokenizer is: 512


Map (num_proc=12):   0%|          | 0/62 [00:00<?, ? examples/s]

success
success
success
successsuccess

success
success
success
success
success
success
success


In [9]:
print(len(datasetDic["test"]))

566


In [ ]:
for i in range(10):
    print(datasetDic["test"][i])

In [11]:
datasetDic.save_to_disk("../processed_datadir/wikitext-103-story-bert-512-wtest")

Saving the dataset (0/2 shards):   0%|          | 0/231188 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7151 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/566 [00:00<?, ? examples/s]